In [3]:
# Install required packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install gensim --user

  Using cached https://files.pythonhosted.org/packages/62/19/8ecba86351de0eacb9baf1cc49ba86315cd91bc672acd74d6e4e709eb482/gensim-3.6.0-cp36-cp36m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ff/17/5cbb026005115301a8fb2f9b0e3e8d32313142fe8b617070e7baad20554f/requests-2.20.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1e/a2/86ac40bc8e091d18c4af321972a0957f387622d13c8e9e618c8a12c22ac1/boto3-1.9.44-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/62/00/ee1d7de624db8ba7090d1226aebefab96a2c71cd5cfa7629d6ad3f61b79e/urllib3-1.24.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/56/9d/1d02dd80bc4cd955f98980f28c5ee2200e1209292d5f9e9cc8d030d18655/certifi-2018.10.15-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e5/14/fca2c32c33cc6321c8820c6c4193131438a1055c95b6951003ebd225f03b

In [2]:
import gensim
import gzip
import urllib
import logging


In [7]:
# Prepare dataset
def prepare_sparql_log_string(line):
    str_line = str(line)
    #print(str_line)
    params = str_line.split()
    url = "http://example.org" + params[5][1:-4]
    qs = urllib.parse.urlparse(url)[4]
    q = urllib.parse.parse_qs(qs)
    #print(q)
    if 'query' in q:
        return q['query'][0]
    
# Preprocess
def preprocess_dataset(dataset):
    for i, line in enumerate(dataset):
        dataset[i] = gensim.utils.simple_preprocess(line)
    
dataset = []
input_file = "data/access.log-20100430.gz"
with gzip.open (input_file, 'rb') as f:
    for i,line in enumerate(f):
        entry = prepare_sparql_log_string(line)
        if entry:
            dataset.append(entry)
            
        
logging.info("Prepared correctly {} queries".format(len(dataset)))

preprocess_dataset(dataset)

model = gensim.models.Word2Vec(
        dataset,
        size=150,
        window=10,
        min_count=2,
        workers=10)
model.train(dataset, total_examples=len(dataset), epochs=10)


(1036504, 3759890)

In [25]:
# Testing the model
w1 = ["order"]

model.wv.most_similar(positive=w1, topn=6)
#model.wv.vocab

[('offset', 0.4479193091392517),
 ('en', 0.4122276306152344),
 ('lang', 0.398708313703537),
 ('limit', 0.34722229838371277),
 ('partner_dance', 0.3251117169857025),
 ('langmatches', 0.28163447976112366)]